In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import random
from datetime import date, timedelta
from snowflake.snowpark.functions import col, lit

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
brands          = ["Callaway", "Odyssey", "Topgolf", "TravisMathew", "OGIO", "Jack Wolfskin"]
regions         = ["NA", "EMEA", "APAC", "LATAM"]
plan_versions   = ["001", "002", "003"]
product_names   = ["Big Bertha Driver", "MAVRIK Iron", "Chrome Soft Golf Ball", "Odyssey White Hot Putter"]

In [ ]:
def generate_data(num_rows):
    data = []
    start_date = date(2024, 1, 1)
    for _ in range(num_rows):
        brand = random.choice(brands)
        region = random.choice(regions)
        plan_version = random.choice(plan_versions)
        product_name = random.choice(product_names)
        actual_net_sales = round(random.uniform(1000, 5000), 2)
        planned_net_sales = round(actual_net_sales * random.uniform(0.9, 1.1), 2)
        actual_standard_margin = round(actual_net_sales * random.uniform(0.2, 0.4), 2)
        actual_cogs_standard = round(actual_net_sales - actual_standard_margin, 2)
        actual_cogs = actual_cogs_standard
        actual_ga = round(actual_net_sales * random.uniform(0.05, 0.1), 2)
        actual_operating_income = actual_standard_margin - actual_ga
        actual_operating_income_excl_corp = actual_operating_income
        planned_standard_margin = round(planned_net_sales * random.uniform(0.2, 0.4), 2)
        planned_cogs_standard = round(planned_net_sales - planned_standard_margin, 2)
        planned_cogs = planned_cogs_standard
        planned_ga = round(planned_net_sales * random.uniform(0.05, 0.1), 2)
        planned_operating_income = planned_standard_margin - planned_ga
        planned_operating_income_excl_corp = planned_operating_income
        period_year = f"{start_date.year}{start_date.strftime('%b').upper()}"
        period_starting_dt = start_date
        current_mth = 'Y' if start_date == date.today().replace(day=1) else 'N'

        row = {
            "EDAP_BRAND_NAME": brand,
            "BRAND": product_name,
            "REGION": region,
            "DERIVED_REGION": region,
            "REGION_GROUP": region,
            "LATEST_PLAN_VERSION": plan_version,
            "ACTUAL_NET_SALES_USD": actual_net_sales,
            "PLANNED_NET_SALES_USD": planned_net_sales,
            "ACTUAL_STANDARD_MARGIN_USD": actual_standard_margin,
            "ACTUAL_COGS_STANDARD_USD": actual_cogs_standard,
            "ACTUAL_COGS_USD": actual_cogs,
            "ACTUAL_GA_USD": actual_ga,
            "ACTUAL_OPERATING_INCOME_USD": actual_operating_income,
            "ACTUAL_OPERATING_INCOME_EXCL_CORP_USD": actual_operating_income_excl_corp,
            "PLANNED_STANDARD_MARGIN_USD": planned_standard_margin,
            "PLANNED_COGS_STANDARD_USD": planned_cogs_standard,
            "PLANNED_COGS_USD": planned_cogs,
            "PLANNED_GA_USD": planned_ga,
            "PLANNED_OPERATING_INCOME_USD": planned_operating_income,
            "PLANNED_OPERATING_INCOME_EXCL_CORP_USD": planned_operating_income_excl_corp,
            "PERIOD_YEAR": period_year,
            "PERIOD_STARTING_DT": period_starting_dt,
            "CURRENT_MTH": current_mth
        }
        data.append(row)
        start_date += timedelta(days=30)  # Increment by approximately one month
    return data


In [ ]:
num_rows = 10  # Specify the number of rows you want to generate
data = generate_data(num_rows)

df = session.create_dataframe(data)

# Define the target table name
target_table = "TB_COPA_AGGR_SYNTHETIC"

# Write the DataFrame to the table
df.write.mode("overwrite").save_as_table(target_table)

In [ ]:
df = session.table("TB_COPA_AGGR_SYNTHETIC").show()

In [ ]:
SELECT * FROM TB_COPA_AGGR_SYNTHETIC;

# SETUP DATABASE AND TABLE

In [ ]:
CREATE DATABASE IF NOT EXISTS TCBrands;
USE DATABASE TCBrands;
USE SCHEMA PUBLIC;

CREATE TABLE IF NOT EXISTS TB_COPA_AGGR_SYNTHETIC (
    EDAP_BRAND_NAME VARCHAR(10),
    BRAND VARCHAR(12),
    REGION VARCHAR(5),
    DERIVED_REGION VARCHAR(10),
    REGION_GROUP VARCHAR(12),
    LATEST_PLAN_VERSION VARCHAR(3),
    ACTUAL_NET_SALES_USD NUMBER(38,2),
    PLANNED_NET_SALES_USD NUMBER(38,2),
    ACTUAL_STANDARD_MARGIN_USD NUMBER(38,2),
    ACTUAL_COGS_STANDARD_USD NUMBER(31,2),
    ACTUAL_COGS_USD NUMBER(38,2),
    ACTUAL_GA_USD NUMBER(31,2),
    ACTUAL_OPERATING_INCOME_USD NUMBER(38,2),
    ACTUAL_OPERATING_INCOME_EXCL_CORP_USD NUMBER(38,2),
    PLANNED_STANDARD_MARGIN_USD NUMBER(38,2),
    PLANNED_COGS_STANDARD_USD NUMBER(31,2),
    PLANNED_COGS_USD NUMBER(38,2),
    PLANNED_GA_USD NUMBER(31,2),
    PLANNED_OPERATING_INCOME_USD NUMBER(38,2),
    PLANNED_OPERATING_INCOME_EXCL_CORP_USD NUMBER(38,2),
    PERIOD_YEAR VARCHAR(7),
    PERIOD_STARTING_DT DATE,
    CURRENT_MTH VARCHAR(1)
);

USE DATABASE TCBrands;
USE SCHEMA PUBLIC;
CREATE OR REPLACE STAGE TCBRANDS_STAGE;